In [10]:
import cloudscraper
import geopandas as gpd
import pandas as pd

In [11]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [12]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [13]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [14]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [15]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [16]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Campground Data

In [17]:
scraper = cloudscraper.create_scraper()

In [62]:
url = "https://api.arbys.com/arb/web-exp-api/v1/location"
params = {
    "latitude": 39.099724,
    "longitude": -94.578331,
    "radius": 5000,
    "limit": 4000,
    "page": 0,
    "locale": "en-us",
}

headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
    "priority": "u=1, i",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "session-id": "",  # Get from https://www.arbys.com/locations/
    "cookie": "",  # Get from https://www.arbys.com/locations/
    "Referer": "https://www.arbys.com/",
    "Referrer-Policy": "strict-origin-when-cross-origin",
}

In [63]:
r = scraper.get(url=url, params=params, headers=headers)

In [64]:
stores_json = r.json()

In [65]:
store_dicts = []
for store_json in tqdm(stores_json["locations"], desc="Parsing Stores"):
    formatted_store_dict = {
        "id": store_json["id"],
        "STUSPS": store_json["contactDetails"]["address"]["stateProvinceCode"],
        "geometry": Point(
            store_json["details"]["longitude"], store_json["details"]["latitude"]
        ),
    }
    store_dicts.append(formatted_store_dict)

Parsing Stores:   0%|          | 0/3287 [00:00<?, ?it/s]

In [66]:
arbys_locations_gdf = gpd.GeoDataFrame(store_dicts, crs=4326)

In [67]:
arbys_locations_gdf.to_file("data/arbys_locations.gpkg")

In [68]:
arbys_counts_df = (
    pd.DataFrame(arbys_locations_gdf.groupby("STUSPS").size())
    .reset_index()
    .rename(columns={0: "arbys"})
)

## Combine With States

In [69]:
arbys_counts_gdf = states_with_population_df.merge(
    arbys_counts_df, on="STUSPS", how="left"
)

In [70]:
arbys_counts_gdf = arbys_counts_gdf.fillna(0)
arbys_counts_gdf["arbys"] = arbys_counts_gdf["arbys"].astype(int)

In [71]:
arbys_counts_gdf["per_100k"] = arbys_counts_gdf["arbys"] / (
    arbys_counts_gdf["POPULATION"] / 100_000
)
arbys_counts_gdf["per_1m"] = arbys_counts_gdf["arbys"] / (
    arbys_counts_gdf["POPULATION"] / 1_000_000
)

In [72]:
arbys_counts_gdf = arbys_counts_gdf.fillna(0)

In [73]:
arbys_counts_gdf = arbys_counts_gdf[
    ["POPULATION", "geometry", "NAME", "per_100k", "per_1m", "arbys"]
]

In [74]:
arbys_counts_gdf = arbys_counts_gdf.to_crs(9311)
arbys_counts_gdf.to_file(f"data/arbys_per_state.gpkg")